In this notebook, we exemplify the utility of the backtesting engine on the backtesting of the MACD strategy

In [23]:
import pandas as pd 
import talib #Technical Indicator Library


# The MACD's popularity is largely due to its ability to help quickly spot increasing short-term momentum.
# Traders will commonly rely on the default settings of 12- and 26-day periods.
# signal line = MACDsignal
# MACD - signal line = MACDhist
FAST_PERIOD = 12 # Default for traders
SLOW_PERIOD = 26 # Default for traders
SIGNAL_PERIOD = 9 # Default for traders
NEUTRAL_THRESHOLD = 0.1 # The difference between signal line and MACD has to be at least this big to provoke a buy or sell signal

class MACD:

    def __init__(self, df):
        self.df = df.copy(deep=False)
        MACD, MACDsignal, MACDhist = talib.MACD(df['Close'], fastperiod=FAST_PERIOD, slowperiod=SLOW_PERIOD, signalperiod=SIGNAL_PERIOD)
        self.MACD = MACD 
        self.MACDsignal = MACDsignal
        self.MACDhist = MACDhist

    def makeRecommendation(self):
        x = self.MACDhist.iloc[-1] #Most recent MACDhisst 
        if x > NEUTRAL_THRESHOLD:
            return 0 # Bullish 
        elif x < -NEUTRAL_THRESHOLD:
            return 1 # Bearish
        else: 
            return 2 #Netural

In [24]:
#!pip install yfinance
import pandas as pd
import numpy as np
import yfinance as yf
import datetime as dt
from pandas_datareader import data as pdr

def data_strat(stocks, start_date, end_date = []):
    df_data = pd.DataFrame()
    df_data_MACD = pd.DataFrame()
    for stock_name in stocks:
        yf.pdr_override()
        
        # First we compute the data with just the trading dates:

        # Start date:
        start_year, start_month, start_day = start_date
        start = dt.datetime(start_year,start_month,start_day)

        # End date
        if end_date: 
            end_year, end_month, end_day = end_date
            end = dt.datetime(end_year,end_month,end_day)
        else: # If no data is passed to end_date, we assume it is now 
            end = dt.datetime.now()

        df=pdr.get_data_yahoo(stock_name,start,end)

        # To change index from date to num:
        df = df.reset_index()
        
        # Drop the colums we are not going to use:
        df = df.drop("High",axis=1)
        df = df.drop("Low",axis=1)
        df = df.drop("Volume",axis=1)
        df = df.drop("Open",axis=1)
        df = df.drop("Close",axis=1) # We don't use the Close Price 
        df = df.drop("Adj Close",axis=1) # Nor do we use the Adj Close Price 
        
        
        
        if not df_data.columns.to_list():
            df_data = df
        else:
            df_data = pd.merge(df_data, df, on="Date")
            
        
        # Then we compute the data that needs to be passed to MACD (180 days before). We use 360 becasue 180 natural days 
        # are not 180 trading days but 360 natural days cover 180 trading days that is what we need
        start_MACD = dt.datetime(start_year,start_month,start_day) - dt.timedelta(days=360) # To cover

        # End date
        if end_date: 
            end_year, end_month, end_day = end_date
            end = dt.datetime(end_year,end_month,end_day)
        else: # If no data is passed to end_date, we assume it is now 
            end = dt.datetime.now()

        df_MACD=pdr.get_data_yahoo(stock_name,start_MACD,end)

        # To change index from date to num:
        df_MACD = df_MACD.reset_index()
        
        # Drop the colums we are not going to use:
        df_MACD = df_MACD.drop("High",axis=1)
        df_MACD = df_MACD.drop("Low",axis=1)
        df_MACD = df_MACD.drop("Open",axis=1)
        df_MACD = df_MACD.drop("Volume",axis=1)
        df_MACD = df_MACD.drop("Close",axis=1) # We don't use the Close Price 
        # df_MACD = df_MACD.drop("Adj Close",axis=1) # We now need the Adj Close
        
        df_MACD = df_MACD.rename(columns={"Adj Close":f"{stock_name} Adj Close"})
        
        if not df_data_MACD.columns.to_list():
            df_data_MACD = df_MACD
        else:
            df_data_MACD = pd.merge(df_data_MACD, df_MACD, on="Date")
        
    
    return df_data, df_data_MACD
    
# load_data(["AAPL","AMZN","TSLA"],[2018,1,1]).head() # Example

In [25]:
def MACD_preds(stocks,start_date,end_date=[]):
    df_data, df_MACD = data_strat(stocks,start_date,end_date)
    
    df_preds = df_data.copy()

    for stock in stocks:
        
        for index, row in df_data.iterrows():
            temp_df = df_MACD[df_MACD['Date'] <= row["Date"]] # We use all the days until the date in which to make the pred
            pred = MACD(df=temp_df.rename(columns={f"{stock} Adj Close":"Close"})).makeRecommendation()

            if pred == 0:
                df_preds.loc[index,[f"{stock} Prediction",f"{stock} Strength"]] = [1,100]
            elif pred == 1:
                df_preds.loc[index,[f"{stock} Prediction",f"{stock} Strength"]] = [-1,100]
            elif pred == 2:
                df_preds.loc[index,[f"{stock} Prediction",f"{stock} Strength"]] = [0,100]

    return df_preds

In [26]:
stocks = ["AMZN","AAPL"]
start_date = [2018,1,1]
end_date = [2020,1,1]
initial_capital = 0
interest_rate = 0.0487

preds_df = MACD_preds(stocks,start_date=start_date,end_date=end_date)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [29]:
preds_df

,Date,AMZN Prediction,AMZN Strength,AAPL Prediction,AAPL Strength
0,2018-01-02,-1.0,100.0,-1.0,100.0
1,2018-01-03,0.0,100.0,0.0,100.0
2,2018-01-04,0.0,100.0,0.0,100.0
3,2018-01-05,0.0,100.0,0.0,100.0
4,2018-01-08,1.0,100.0,0.0,100.0
...,...,...,...,...,...
498,2019-12-24,1.0,100.0,1.0,100.0
499,2019-12-26,1.0,100.0,1.0,100.0
500,2019-12-27,1.0,100.0,1.0,100.0
501,2019-12-30,1.0,100.0,1.0,100.0


In [28]:
#!pip install import_ipynb
import import_ipynb
from Backtesting_Engine import backtest

data = backtest(stocks,initial_capital,interest_rate,preds_df,start_date,end_date)
data.to_excel("data.xlsx")
data

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

Net profit in the selected period: 1428.626$


,Date,AMZN Open,AMZN Pred num,AMZN Nº of shares,AAPL Open,AAPL Pred num,AAPL Nº of shares,Total Nº of shares,Daily interest ($),Total interest ($),Total money owed ($),Invested money ($),Tomorrow's Available money ($),Daily profit ($),Net profit ($),Capital ($),AMZN Tom's - Tod's open ($),AAPL Tom's - Tod's open ($)
0,2018-01-02,58.599998,-1.0,0,42.540001,-1.0,0,0,0.000,0.000,0.000,0.000,0.0,0.000,0.000,0.000,0.815,0.592
1,2018-01-03,59.415001,0.0,0,43.132500,0.0,0,0,0.000,0.000,0.000,0.000,0.0,0.000,0.000,0.000,0.835,0.002
2,2018-01-04,60.250000,0.0,0,43.134998,0.0,0,0,0.000,0.000,0.000,0.000,0.0,0.000,0.000,0.000,0.625,0.225
3,2018-01-05,60.875500,0.0,0,43.360001,0.0,0,0,0.000,0.000,0.000,0.000,0.0,0.000,0.000,0.000,0.924,0.228
4,2018-01-08,61.799999,1.0,1,43.587502,0.0,0,1,0.000,0.000,61.800,62.845,0.0,1.045,1.045,1.045,1.045,0.050
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
498,2019-12-24,89.690498,1.0,39,71.172501,1.0,62,101,1.254,259.371,6651.356,7926.680,0.0,14.801,1275.324,1275.324,0.360,0.033
499,2019-12-26,90.050499,1.0,40,71.205002,1.0,63,103,1.286,260.657,6813.897,8350.980,0.0,261.759,1537.083,1537.083,4.096,1.575
500,2019-12-27,94.146004,1.0,41,72.779999,1.0,64,105,1.317,261.974,6982.140,8473.060,0.0,-46.163,1490.920,1490.920,-0.446,-0.415
501,2019-12-30,93.699997,1.0,42,72.364998,1.0,65,107,1.349,263.323,7149.554,8579.562,0.0,-60.912,1430.008,1430.008,-1.600,0.118
